# [IAPR 2020:][iapr2020] Lab 3 ‒  Classification

**Author:** Quentin Talon, Albéric De Lajarte
**Due date:** 08.05.2020

[iapr2018]: https://github.com/LTS5/iapr-2018

## Extract relevant data
We first need to extract the `lab-03-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [ ]:
import tarfile
import os

data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-03-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)

## Part 1
In this part, we will study classification based on the data available in the Matlab file `classification.mat` that you will under `lab-03-data/part1`.
There are 3 data sets in this file, each one being a training set for a given class.
They are contained in variables `a`, `b` and `c`.

**Note**: we can load Matlab files using the [scipy.io] module.

[scipy.io]: https://docs.scipy.org/doc/scipy/reference/io.html

In [ ]:
import scipy.io

data_part1_path = os.path.join(data_base_path, data_folder, 'part1', 'classification.mat')
matfile = scipy.io.loadmat(data_part1_path)
a = matfile['a']
b = matfile['b']
c = matfile['c']

print(a.shape, b.shape, c.shape)

In [ ]:
#Dataset visualization
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from numpy import linalg as LA
import numpy as np

plt.figure(figsize=(20,10))
ax = plt.gca()

#Plot dataset
plt.scatter(a[:,0], a[:,1], marker= '+')
plt.scatter(b[:,0], b[:,1], marker= 'o')
plt.scatter(c[:,0], c[:,1], marker= ',')

#Compute mean of each dataset
mu_a = [np.mean(a[:,0]), np.mean(a[:,1])]
mu_b = [np.mean(b[:,0]), np.mean(b[:,1])]
mu_c = [np.mean(c[:,0]), np.mean(c[:,1])]

#Compute variance of each dataset
var_a = np.cov(a, rowvar=False)
var_b = np.cov(b, rowvar=False)
var_c = np.cov(c, rowvar=False)

#Plot dataset
plt.scatter(mu_a[0], mu_a[1], marker= '+', color = 'r')
plt.scatter(mu_b[0], mu_b[1], marker= 'o', color = 'r')
plt.scatter(mu_c[0], mu_c[1], marker= ',', color = 'r')

#Plot covariance ellipse
w,v = LA.eig(var_a)
ax.add_patch(Ellipse(xy=mu_a, width=4*np.sqrt(w[0]), height=4*np.sqrt(w[1]), edgecolor='r', fc='None', lw=2, angle=np.degrees(np.arctan2(w[1], w[0])) ))
w,v = LA.eig(var_b)
ax.add_patch(Ellipse(xy=mu_b, width=4*np.sqrt(w[0]), height=4*np.sqrt(w[1]), edgecolor='r', fc='None', lw=2, angle=np.degrees(np.arctan2(w[1], w[0])) ))
w,v = LA.eig(var_c)
ax.add_patch(Ellipse(xy=mu_c, width=4*np.sqrt(w[0]), height=4*np.sqrt(w[1]), edgecolor='r', fc='None', lw=2, angle=np.degrees(np.arctan2(w[0], w[1])) ))


plt.show()

### 1.1 Bayes method
Using the Bayes method, give the analytical expression of the separation curves between those three classes.
Do reasonable hypotheses about the distributions of those classes and estimate the corresponding parameters based on the given training sets.
Draw those curves on a plot, together with the training data.
For simplicity reasons, round the estimated parameters to the closest integer value.

*Add your implementation and discussion*

##### Analytical expression
We compute all quadrics between all the classes. Then, for a given point $x$ we see where it belongs.
- A or B -> $x\notin B$
- A or C -> $x\notin C$
- B or C -> $x\notin C$

If a point doesnt not belong to B and C, we conclue it's in A.

In [ ]:
# Create 2D space
x = np.linspace(-20, 20, 50)
y = np.linspace(-10, 10, 50)
X, Y = np.meshgrid(x, y)

# Analytical expression of Bayesian discriminant function with diagonal covariance matrix and no constant 
def discriminant_bayesian(mu, var):
    C = np.transpose(mu)@LA.inv(var)@mu/(-2)
    G = (var[1,1]*X**2 + var[0,0]*Y**2)/(-2*var[1,1]*var[0,0]) + (var[1,1]*X*mu[0] + var[0,0]*Y*mu[1])/(var[1,1]*var[0,0]) + C
    return G

G_a = discriminant_bayesian(mu_a, var_a)
G_b = discriminant_bayesian(mu_b, var_b)
G_c = discriminant_bayesian(mu_c, var_c)

fig, ax = plt.subplots(2, 3, figsize = (15, 10))

# Plot the function with the dataset
ax[0][0].pcolor(X, Y,G_a)
ax[0][0].scatter(a[:,0], a[:,1], marker= '+')

ax[0][1].pcolor(X, Y, G_b)
ax[0][1].scatter(b[:,0], b[:,1], marker= 'o')

ax[0][2].pcolor(X, Y, G_c)
ax[0][2].scatter(c[:,0], c[:,1], marker= ',')

# Plot difference of bayesian function and invert it to outline near zero term
pos = ax[1][0].pcolor(X, Y, 10/abs(G_a-G_b))
ax[1][0].scatter(a[:,0], a[:,1], marker= '+')
ax[1][0].scatter(b[:,0], b[:,1], marker= 'o')
fig.colorbar(pos, ax = ax[1][0])

ax[1][1].pcolor(X, Y, 10/abs(G_b-G_c))
ax[1][1].scatter(c[:,0], c[:,1], marker= ',')
ax[1][1].scatter(b[:,0], b[:,1], marker= 'o')

ax[1][2].pcolor(X, Y, 10/abs(G_c-G_a))
ax[1][2].scatter(a[:,0], a[:,1], marker= '+')
ax[1][2].scatter(c[:,0], c[:,1], marker= ',')




In [ ]:
# Find contour of near zero term
from skimage.measure import find_contours
G_inter = abs(G_a-G_c)
contour = find_contours(G_inter, 54, fully_connected = "low")[0]
contour = [x[contour[:, 1].astype(int)], y[contour[:, 0].astype(int)]]

# Fit parametric quadratic curve on the contour
contour_parameter = np.polyfit(contour[0], contour[1], 2)
contour_quad = np.poly1d(contour_parameter)

# Plot bayesian function and contour
plt.pcolor(X, Y, G_inter)
plt.colorbar()

plt.plot(contour[0], contour[1], linewidth = 4)
plt.plot(x, contour_quad(x))

In [ ]:
#Compute lines from analytical estimation
g_ab = 0
g_ac = 0
g_bc = 0

In [ ]:
# Simple test with 1D gaussian function
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

t = np.linspace(0, 10, 100)
f1 = gaussian(t, 7, 1)
f2 = gaussian(t, 2, 2)

# plot 2 gaussian and their difference
plt.plot(t, f1)
plt.plot(t, f2)
plt.plot(t, 0.01/abs(f2-f1))

# Note: difference works well, but scaling necessary for visualization, and careful with edge of function

### 1.2 Mahalanobis distance
For classes `a` and `b`, give the expression of the Mahalanobis distance used to classify a point in class `a` or `b`, and verify the obtained classification, in comparison with the "complete" Bayes classification, for a few points of the plane.

*Add your implementation and discussion*

## Part 2
In this part, we aim to classify digits using the complete version of MNIST digits dataset.
The dataset consists of 60'000 training images and 10'000 test images of handwritten digits.
Each image has size 28x28, and has assigned a label from zero to nine, denoting the digits value.
Given this data, your task is to construct a Multilayer Perceptron (MLP) for supervised training and classification and evaluate it on the test images.

Download the MNIST dataset (all 4 files) from http://yann.lecun.com/exdb/mnist/ under `lab-03-data/part2`.
You can then use the script provided below to extract and load training and testing images in Python.

To create an MLP you are free to choose any library.
In case you don't have any preferences, we encourage you to use the [scikit-learn] package; it is a simple, efficient and free tool for data analysis and machine learning.
In this [link][sklearn-example], you can find a basic example to see how to create and train an MLP using [scikit-learn].
Your network should have the following properties:
* Input `x`: 784-dimensional (i.e. 784 visible units representing the flattened 28x28 pixel images).
* 100 hidden units `h`.
* 10 output units `y`, i.e. the labels, with a value close to one in the i-th class representing a high probability of the input representing the digit `i`.

If you need additional examples you can borrow some code from image classification tutorials.
However, we recommend that you construct a minimal version of the network on your own to gain better insights.

[scikit-learn]: http://scikit-learn.org/stable/index.html
[sklearn-example]: http://scikit-learn.org/stable/modules/neural_networks_supervised.html

### 2.1 Dataset loading
Here we first declare the methods `extract_data` and `extract_labels` so that we can reuse them later in the code.
Then we extract both the data and corresponding labels, and plot randomly some images and corresponding labels of the training set.

In [ ]:
import gzip
import numpy as np
import matplotlib.pyplot as plt

def extract_data(filename, image_shape, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(np.prod(image_shape) * image_number)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(image_number, image_shape[0], image_shape[1])
    return data


def extract_labels(filename, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * image_number)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return labels

In [ ]:
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000

data_part2_folder = os.path.join(data_base_path, data_folder, 'part2')

train_images_path = os.path.join(data_part2_folder, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_part2_folder, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(data_part2_folder, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_part2_folder, 't10k-labels-idx1-ubyte.gz')

train_images = extract_data(train_images_path, image_shape, train_set_size)
test_images = extract_data(test_images_path, image_shape, test_set_size)
train_labels = extract_labels(train_labels_path, train_set_size)
test_labels = extract_labels(test_labels_path, test_set_size)

In [ ]:
prng = np.random.RandomState(seed=123456789)  # seed to always re-draw the same distribution
plt_ind = prng.randint(low=0, high=train_set_size, size=10)

fig, axes = plt.subplots(1, 10, figsize=(12, 3))
for ax, im, lb in zip(axes, train_images[plt_ind], train_labels[plt_ind]):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(lb)

### 2.2 MLP
*Add your implementation and discussion*